In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
prefix = "/kaggle/input/original-instant-gratification"

train = pd.read_csv(f"{prefix}/train.csv")
test = pd.read_csv(f"{prefix}/public_test.csv")
submission = pd.read_csv(f"{prefix}/sample_submission.csv")

## 데이터 EDA

In [ ]:
print(train.shape)
print(test.shape)

train.head()

In [ ]:
test.head()

In [ ]:
print(train['target'].value_counts())

In [ ]:
train['target'].cumsum().plot()

In [ ]:
print(train.isnull().sum())
print(train.isnull().sum().sum())

In [ ]:
train.dtypes.value_counts()

In [ ]:
train.select_dtypes(include=["object", "int64"]).head()

In [ ]:
# id, target을 제외한 train 열 선택
train_columns = [col for col in train.columns if col not in ["id", "target"]]
# print(train_columns)
print(train[train_columns].nunique())

In [ ]:
print(np.where(train[train_columns].nunique() < 250000))
print(train_columns[182])

In [ ]:
train["wheezy-copper-turtle-magic"].nunique()

In [ ]:
# wheezy-copper-turtle-magic 열은 고유한 값이 512개 뿐이므로 다른 열에 비해 매우 작으며 인위적인 느낌이 듬 -> 추가적으로 분석
target = "wheezy-copper-turtle-magic"
print(train[target].value_counts().mean())

In [ ]:
train[target].value_counts().rename_axis(target).reset_index(name="counts")

## 수치형(float) 변수 확인

In [ ]:
train_columns = [
    col for col in train.columns if col not in ["id", "target", "wheezy-copper-turtle-magic"]
]
describe_train = train[train_columns].describe().T.drop("count", axis=1)
cmap = sns.diverging_palette(5, 250, as_cmap=True)
describe_train.T.style.background_gradient(cmap, axis=1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
train_columns = [
    col for col in train.columns if col not in ["id", "target"]
]
target_0_df = train.loc[train["target"] == 0]
target_1_df = train.loc[train["target"] == 1]

plt.figure(figsize=(16, 12))
for index, name in enumerate(train_columns[:25]):
    plt.subplot(5, 5, index + 1)
    sns.distplot(target_0_df[name])
    sns.distplot(target_1_df[name])
    plt.title(name)
    plt.xlabel("")
    plt.legend(["0", "1"])
    
plt.tight_layout()
plt.show()

## EDA 과정 정리
### 데이터의 형식 확인
- 열 안에 List나 Json의 형태가 있을 수 있고 그 안의 어떤 정보를 추출할 수 있는지 나열해 보는 것도 좋음
    
### 정답 값의 비율 확인
- 정답의 불균형 여부를 확인한 뒤 추후 고려하여 모델을 구성해야 함

### 정답 값 누출 확인
- 주최자가 의도하지 않은, 문제에 발생한 누출은 암묵적으로 공유되는 문화, 빠르게 캐치한다면 discussion이나 노트북에서 좋은 메달을 받을 수 있음

### NULL값 확인
- 알고리즘에 따라 NULL 값을 필수로 채워야 할 수도 있음
- 단순히 -999와 같은 값으로 채우기보다는 왜 NULL이 생성되었고 어떤 의미인지 먼저 파악하는 것이 좋음

### 범주형 변수와 수치형 변수를 나누기
- 각 특성에 따라 전처리하는 방법이 다르기 때문에 EDA 단계에서 미리 구분해두는 것이 좋음

### Feature는 정답값과 연관지어 생각하기
- 특정 범주형 변수에 정답 값의 분포가 몰려 있거나, 수치형 변수의 분포를 살펴봤을 때 정답 값별로 분포가 다르면 좋은 변수가 될 가능성이 큼